In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_path = "/content/drive/MyDrive/Data Science/Project-44 Petrol Price Forecasting Using Auto Keras/train_data.csv"
test_path = "/content/drive/MyDrive/Data Science/Project-44 Petrol Price Forecasting Using Auto Keras/test_data.csv"

In [ ]:
df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
df.shape, test_df.shape

In [ ]:
df.head()

In [ ]:
# checking Null values

df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.rename(columns={'Petrol (USD)': 'Petrol'}, inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Date', y='Petrol', s=50 , label='Price')
plt.show()

In [ ]:
# dropping outlayers

df = df[df['Petrol'] < 150]

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Date', y='Petrol', s=50 , label='Price')
plt.show()

In [ ]:
data = df['Petrol'].values

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
data = scaler.fit_transform(data.reshape(-1, 1))

In [ ]:
data[:5]

In [ ]:
training_size = int(len(data)* 0.65)
testing_size = len(data) - training_size

In [ ]:
training_data = data[0:training_size]
testing_data = data[training_size:]

In [ ]:
training_data.shape, testing_data.shape

In [ ]:
def create_data(dataset, time_step=0):
  dataX, dataY = [], []
  for i in range(len(dataset)-time_step - 1):
    a = dataset[i: i+time_step, 0]
    dataX.append(a)
    dataY.append(dataset[i])
    

  return np.array(dataX), np.array(dataY) 
    

In [ ]:
x_train, y_train = create_data(training_data, 100)
x_test, y_test = create_data(testing_data, 100)

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
# Reshaping Data 
X_train = x_train.reshape(424, 100, 1)
X_test = x_test.reshape(183, 100, 1)

In [ ]:
# import build RNN Model (LSTM)
import tensorflow as tf
import keras

In [ ]:
model = keras.models.Sequential()

In [ ]:
# adding layers

model.add(keras.layers.LSTM(50, return_sequences=True, input_shape=(100, 1)))
model.add(keras.layers.LSTM(50, return_sequences=True))
model.add(keras.layers.LSTM(50))
model.add(keras.layers.Dense(1))

In [ ]:
# Compilte the Model

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# training the model
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

In [ ]:
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

In [ ]:
train_pred = scaler.inverse_transform(train_pred)
test_pred = scaler.inverse_transform(test_pred)

## Arima model

In [ ]:
# ! pip install statsmodels

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
ar_model = ARIMA(df['Petrol'].values, order=(5, 1, 0))

In [ ]:
model_ = ar_model.fit()

In [ ]:
output= model_.forecast(20)

In [ ]:
output

## Auto Keras

In [ ]:
# ! pip install git+https://github.com/keras-team/keras-tuner.git
# ! pip install autokeras

In [ ]:
import autokeras as ak

In [ ]:
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=3
)

In [ ]:
reg.fit(x_train, y_train)

In [ ]:
mae, _ = reg.evaluate(x_test, y_test)

In [ ]:
print(mae, _)

In [ ]:
model = reg.export_model()

In [ ]:
model.summary()